In [203]:
#Your task is to create a Python script that analyzes the records to calculate each of the following:
#
#* The total number of months included in the dataset.
#* The net total amount of Profit/Losses over the entire period.
#* The average of the changes in Profit/Losses over the entire period.
#* The greatest increase in profits (date and amount) over the entire period.
#* The greatest decrease in losses (date and amount) over the entire period.
#
#Your resulting analysis should look similar to the following:
#
#  ```text
#  Financial Analysis
#  ----------------------------
#  Total Months: 86
#  Total: $38382578
#  Average  Change: $-2315.12
#  Greatest Increase in Profits: Feb-2012 ($1926159)
#  Greatest Decrease in Profits: Sep-2013 ($-2196167)
#  ```
###########

##
## IMPORTS
##

from pathlib import Path
import csv

#import babel.numbers
#import decimal
#babel.numbers.format_currency( decimal.Decimal( "188518982.18" ), "AUD" )

##
## FUNCTIONS
##

# funtion uses a sniffer to determine whether a header row exists.
def has_header(file_in):
    with open(budget_data_file,'rt',closefd=True) as csv_file:
        # do we have a header?
        sample = csv_file.read(128)
        hh_tf = csv.Sniffer().has_header(sample)
    return hh_tf

def perform_analysis(file_in,count_pos,num_elmnts,analysis_output):
    # start processing the file
    with open(budget_data_file,'rt',closefd=True) as csv_file:
        # initialise variables
        csv_generator = csv.reader(csv_file, delimiter=',')
            #AA remove
            #
            #csv_list = list(csv.reader(csv_file, delimiter=','))
            #csv_processor = csv.reader(csv_file, delimiter=',')
            #num_elmnts = len(next(csv_processor))
            #num_elmnts = num_elmnts
        row_count = count_pos
        data_sum = 0
        diff_sum = 0
        data_avg = 0
        prev_amnt = 0
        diff_prd = 0
        month_min = ''
        data_min = 0
        diff_min = 0
        month_max = ''
        data_max = 0
        diff_max = 0
        
        # header? skip row and reset counter
        if count_pos < 0:
            next(csv_generator)
            row_count += 1
            
        # perform analysis
        for row in csv_generator:
            # iterate and count months
            row_count += 1
            
            # sum each ... the pos and neg amounts will balance out
            data_sum += int(row[num_elmnts-1])
            
            # record the diff
            diff_prd = int(row[num_elmnts-1])-prev_amnt
            
            # determine mins and maxs
            if prev_amnt == 0:
                month_min = row[0]
                diff_min = 0
                diff_max = 0
                month_max = row[0]
                diff_prd = 0
                data_avg = 0
            elif int(row[num_elmnts-1]) < data_min:
                # note the latest lowest amount, month and diff
                month_min = row[0]
                data_min = int(row[num_elmnts-1])
                diff_min = int(row[num_elmnts-1])-prev_amnt
            elif int(row[num_elmnts-1]) > data_max:
                # note the latest highest amount, month and diff
                month_max = row[0]
                data_max = int(row[num_elmnts-1])
                diff_max = int(row[num_elmnts-1])-prev_amnt
            
            # record the diff
            #diff_prd = int(row[num_elmnts-1])-prev_amnt
            
            # sum diff
            diff_sum += diff_prd
            #print(f"data: {row[num_elmnts-1]}")
            
            # average of the diff
            #data_avg = diff_sum/row_count
            
            # record the amount for the next loop
            prev_amnt = int(row[num_elmnts-1])
            
            #track ...
            #print(f"avg: {round(data_avg,2)}, diff {diff_prd}, mins({diff_min},{data_min},{month_min}), max({diff_max},{data_max},{month_max}) row: {row}")
            #print(f"avg1: {round(data_avg_1,2)}, avg2: {round(data_avg_2,2)}, avg3: {round(data_avg_3,2)}, avg4: {round(data_avg_4,2)} row: {row}")
            #print(f"diff {diff_prd}, d_sum {diff_sum}, avg: {round(data_avg,2)}, cnt: {row_count}")
            
        data_avg = round(diff_sum/(row_count-1),2)
        
        with open(analysis_output,'w',closefd=True) as fileout:
            print("Financial Analysis")
            fileout.write("Financial Analysis\n")
            print("----------------------------------------------------")
            fileout.write("----------------------------------------------------\n")
            print(f"Total Months                : {row_count}")
            fileout.write(f"Total Months                : {row_count}\n")
            print(f"Total                       : ${data_sum}")
            fileout.write(f"Total                       : ${data_sum}\n")
            print(f"Average Change              : ${data_avg}")
            fileout.write(f"Average Change              : ${data_avg}\n")
            print(f"Greatest Increase in Profits: ${diff_max} in {month_max}")
            fileout.write(f"Greatest Increase in Profits: ${diff_max} in {month_max}\n")
            print(f"Greatest Decrease in Profits: ${diff_min} in {month_min}")
            fileout.write(f"Greatest Decrease in Profits: ${diff_min} in {month_min}\n")
    return

##
## MAIN
##

# Set the file name for input/output
budget_data_file = "./Resources/budget_data.csv"
analysis_output = "./budget_analysis.txt"

# if there is a header, read it, store it, and count its #cols.
if has_header:
    # extract the header
    with open(budget_data_file,'rt',closefd=True) as csv_file:
        csv_header = next(csv.reader(csv_file, delimiter=','))
        csv_cols   = len(csv_header)
        #print(f"header: {csv_header} #cols: {csv_head_cols}") # AA remove
    # count numrows without counting the header
    start_count = -1
else:
    # there is no header, continue anyway rockstar
    # determine cols for first row
    with open(budget_data_file,'rt',closefd=True) as csv_file:
        csv_first_row = next(csv.reader(csv_file, delimiter=','))
        csv_cols      = len(csv_first_row)
        #print(f"first: {csv_first_row} #cols: {csv_head_cols}") # AA remove
    # count numrows normally
    start_count = 0

#AA removeprint(f"calling perform_analysis({budget_data_file},{start_count},{csv_cols})") # AA remove
#AA removeprint("----------------------------------------------------------")

# perform the analysis
perform_analysis(budget_data_file,start_count,csv_cols,analysis_output)




Financial Analysis
----------------------------------------------------
Total Months                : 86
Total                       : $38382578
Average Change              : $-2315.12
Greatest Increase in Profits: $1926159 in Feb-2012
Greatest Decrease in Profits: $-2196167 in Sep-2013
